# Some text

In [ ]:
!pip install chart-studio

In [ ]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [ ]:
import chart_studio.plotly as py
import numpy as np
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import  Contours, Histogram2dContour, Marker, Scatter

configure_plotly_browser_state()

init_notebook_mode(connected=False)

x = np.random.randn(2000)
y = np.random.randn(2000)
iplot([Histogram2dContour(x=x, y=y, contours=Contours(coloring='heatmap')),
       Scatter(x=x, y=y, mode='markers', marker=Marker(color='white', size=3, opacity=0.3))], show_link=False)



In [ ]:
import boto3
import base64
from botocore.exceptions import ClientError
import json
import chart_studio.plotly as py
import plotly.graph_objs as go


def get_secret():
    secret_name = "202012-awsmap"
    region_name = "us-west-2"

    session = boto3.session.Session()
    client = session.client(service_name='secretsmanager',   region_name=region_name)
    get_secret_value_response = client.get_secret_value(SecretId=secret_name)

    secret_value_json_object = json.loads(get_secret_value_response['SecretString'])
    return secret_value_json_object

secret_value_json_object = get_secret()

plotly_username = secret_value_json_object['plotly_username']
plotly_api_key = secret_value_json_object['plotly_api_key']
mapbox_access_token = secret_value_json_object['mapbox_access_token']
py.sign_in(plotly_username, plotly_api_key)


In [ ]:
import boto3
from boto3.dynamodb.conditions import Key, Attr

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('saubury-001')

fe = Key('time').between('2020-12-27T22:45', '2020-12-28T05:00');
response = table.scan(FilterExpression=fe)


items = response['Items']
lat_array = []
lon_array = []

for item in items:
    lat_array.append(item['Latitude'])
    lon_array.append(item['Longitude'])

data = [
    go.Scattermapbox(
        lat=lat_array,
        lon=lon_array,
        mode='markers',
        marker=dict(
            size=14
        ),
        text=['foo', 'bar'],
    )
]

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=-33.805,
            lon=151.0816
        ),
        pitch=0,
        zoom=8
    ),
)

fig = dict(data=data, layout=layout)

py.iplot(fig, filename='Montreal Mapbox')